Rossler performance experiments

In [1]:
import numpy as np
import torch
import sys
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from IPython.display import clear_output

sys.path.append('../')
from source.utils import compare_graphs
import source.utils as utils
import source.NMC as models
import importlib

## SVAM

In [19]:
# LiNGAM / SVAM performance with sparse data
import warnings
warnings.filterwarnings("ignore")
for p in [10,50]:
    perf = []
    for i in range(20):
        # Simulate data
        T = 1000
        num_points = T
        data, GC = utils.simulate_rossler(p=p, a=0, T=T, delta_t=0.1, sd=0.05, burn_in=0, sigma=0.0)

        # format for NeuralODE
        data = torch.from_numpy(data[:, None, :].astype(np.float32))

        from source.lingam import lingam_method
        importlib.reload(utils)
        graph = lingam_method(data.squeeze().detach())
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('Means and standard deviations for TPR, FDR and AUC with', p, 'dimensions')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

Means and standard deviations for TPR, FDR and AUC with 10 dimensions
[0.758 0.567 0.484] [0.048 0.039 0.028]
Means and standard deviations for TPR, FDR and AUC with 50 dimensions
[0.566 0.677 0.499] [0.093 0.061 0.027]


# DCM

In [22]:
# DCM performance with sparse data
for p in [10,50]:
    perf = []
    for i in range(10):
        # Simulate data
        T = 1000
        num_points = T
        data, GC = utils.simulate_rossler(p=p, a=0, T=T, delta_t=0.1, sd=0.05, burn_in=0, sigma=0.0)

        from source.DCM import DCM_full
        graph = DCM_full(data,lambda1=0.001, s=4, w_threshold = 0.1)
        #plt.matshow(abs(graph),cmap='Reds')
        #plt.colorbar()
        #plt.show()
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('Means and standard deviations for TPR, FDR and AUC with', p, 'dimensions')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

Means and standard deviations for TPR, FDR and AUC with 10 dimensions
[0.842 0.323 0.911] [0.    0.161 0.006]
Means and standard deviations for TPR, FDR and AUC with 50 dimensions
[0.97  0.689 0.983] [2.220e-16 5.757e-02 3.826e-04]


# PCMCI

In [21]:
# pcmci performance with sparse data
for p in [10,50]:
    perf = []
    for i in range(5):
        # Simulate data
        T = 1000
        num_points = T
        data, GC = utils.simulate_rossler(p=p, a=0, T=T, delta_t=0.1, sd=0.05, burn_in=0, sigma=0.0)
        
        from source.pcmci import pcmci
        importlib.reload(utils)
        graph = pcmci(data)
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('TPR, FDR and AUC with', p, 'dimensions')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

TPR, FDR and AUC with 10 dimensions
[0.695 0.736 0.619] [0.102 0.04  0.061]
TPR, FDR and AUC with 50 dimensions
[0.824 0.916 0.726] [0.04  0.008 0.029]


## NGM

In [23]:
# NGM performance with sparse data
import warnings
warnings.filterwarnings("ignore")
for p in [10,50]:
    perf = []
    for i in range(1):
        # Simulate data
        T = 1000
        num_points = T
        data, GC = utils.simulate_rossler(p=p, a=0, T=T, delta_t=0.1, sd=0.05, burn_in=0, sigma=0.0)

        # format for NeuralODE
        data = torch.from_numpy(data[:, None, :])

        import source.NMC as models
        func = models.MLPODEF(dims=[p, 12, 1],GL_reg=0.1)
        
        # GL training
        models.train(func,data,n_steps=2000,plot = False, plot_freq=20)
        # AGL training
        #weights = func.group_weights()
        #func.GL_reg *= (1 / weights)
        #func.reset_parameters()
        #models.train(func,data,n_steps=1000,plot = True, plot_freq=20)
        graph = func.causal_graph(w_threshold=0.1)
        perf.append(utils.compare_graphs(GC, graph)) # tpr, fdr

    print('Means and standard deviations for TPR, FDR and AUC with', p, 'dimensions')
    print(np.mean(np.reshape(perf,(-1, 3)),axis=0), np.std(np.reshape(perf,(-1, 3)),axis=0) )     

Means and standard deviations for TPR, FDR and AUC with 10 dimensions
[0.895 0.    0.947] [0. 0. 0.]
Means and standard deviations for TPR, FDR and AUC with 50 dimensions
[0.97  0.127 0.984] [0. 0. 0.]
